## 1. 라이브러리 로드

In [9]:
import os
import cv2
import json
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage

## 2. 경로 설정

In [10]:
input_image_path = "C:\\Users\\rhoms\\businside\\bus_passenger\\Validation\\img_allInOne"
input_json_path = "data/val.json"
output_image_path = "data/aug_val_img"
output_json_path = "data/json"
output_json_path = os.path.join(output_json_path, "aug_val.json")

# JSON 파일에서 COCO 데이터 불러오기
with open(input_json_path, "r") as f:
    coco_data = json.load(f)

## 3. 이미지, json 파일 잘 로드되었는지 확인하기

In [11]:
# 이미지 정보 가져오기
image_info = coco_data["images"][0]

# 이미지 파일 경로 생성
image_path = os.path.join(input_image_path, image_info["file_name"])

# 이미지 파일 읽어오기
# Image Shape: (1080, 1920, 3) 이렇게 출력나와야 함
if os.path.exists(image_path):
    image = cv2.imread(image_path)
    if image is not None:
        print("Image Shape:", image.shape)
    else:
        print("Failed to read image:", image_path)
else:
    print("Image file not found:", image_path)


Image file not found: c:\Users\rhoms\businside\bus_passenger\Validation\img_allInOne\[district]leave_634C_96.jpg


In [12]:
#print("coco data: ", coco_data)

In [13]:
image_info = coco_data["images"][0]

image_path = os.path.join(input_image_path, image_info["file_name"])
image = cv2.imread(image_path)
print(image_info)


{'height': 1080, 'width': 1920, 'id': 1, 'file_name': 'c:\\Users\\rhoms\\businside\\bus_passenger\\Validation\\img_allInOne\\[district]leave_634C_96.jpg'}


In [14]:
print("Image Path:", image_path)
image = cv2.imread(image_path)
print("Image:", image)
# 이미지 3차원으로 나와야 함

Image Path: c:\Users\rhoms\businside\bus_passenger\Validation\img_allInOne\[district]leave_634C_96.jpg
Image: None


In [15]:
# print("Image Width:", image.shape[1])
# print("Image Height:", image.shape[0])

## 4. Augmentation

In [16]:
# 총 7개 aug 기법 적용하기
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # 좌우 반전
    iaa.GaussianBlur(sigma=(0, 1.0)),  # 가우시안 블러
    iaa.MultiplyBrightness((0.7, 1.3)),  # 밝기 조절
    iaa.ChangeColorspace(from_colorspace="RGB", to_colorspace="HSV"),
    iaa.WithChannels(0, iaa.Add((-30, 30))),  # 색상 조정
    iaa.ChangeColorspace(from_colorspace="HSV", to_colorspace="RGB"),
    iaa.AdditiveGaussianNoise(scale=(0, 0.1 * 255))  # 가우시안 노이즈 추가
])

# 이미지와 바운딩 박스 정보에 aug 적용하고 COCO 데이터 형식으로 수정
for image_info in coco_data["images"]:
    image_path = os.path.join(input_image_path, image_info["file_name"])
    image = cv2.imread(image_path)
    
    # 이미지가 None이라면 스킵
    if image is None:
        continue
    
    # 해당 이미지에 대한 바운딩 박스 정보 가져오기
    annotations = [ann for ann in coco_data["annotations"] if ann["image_id"] == image_info["id"]]
    
    # annotations 리스트가 비어있는 경우 스킵
    if not annotations:
        continue
    
    # 모든 증강 기법 순차적으로 적용
    augmented_images = []
    augmented_annotations = []
    
    for aug in seq:
        augmented_annotations_per_aug = []  # 현재 증강 기법에 대한 바운딩 박스 정보 저장 리스트
        for ann in annotations:
            bbox = BoundingBox(x1=ann["bbox"][0], y1=ann["bbox"][1], x2=ann["bbox"][0] + ann["bbox"][2], y2=ann["bbox"][1] + ann["bbox"][3])
            augmented_image, augmented_bbs = aug(image=image, bounding_boxes=[bbox])
            augmented_images.append(augmented_image)
            augmented_annotations_per_aug.append(augmented_bbs[0])
        
        augmented_annotations.append(augmented_annotations_per_aug)
    
    # 각 증강된 이미지와 바운딩 박스 정보에 대해 저장
    for i, augmented_image in enumerate(augmented_images):
        if i < len(augmented_annotations):  # 인덱스 범위를 초과하지 않도록 체크
            output_image_file = os.path.basename(image_info["file_name"]).replace(".jpg", f"_aug_{i}.jpg")
            output_image_path_full = os.path.join(output_image_path, output_image_file)
            cv2.imwrite(output_image_path_full, augmented_image)
        
            for j, augmented_bbs in enumerate(augmented_annotations[i]):
                new_image_info = image_info.copy()
                new_image_info["file_name"] = output_image_file
                new_image_info["id"] = len(coco_data["images"]) + 1
            
                new_ann = annotations[j].copy()
                new_ann["bbox"] = [augmented_bbs.x1, augmented_bbs.y1, augmented_bbs.x2 - augmented_bbs.x1, augmented_bbs.y2 - augmented_bbs.y1]
                new_ann["id"] = len(coco_data["annotations"]) + 1 + j
                new_ann["image_id"] = new_image_info["id"]
            
                coco_data["images"].append(new_image_info)
                coco_data["annotations"].append(new_ann)

# 수정된 COCO 데이터 형식을 JSON 파일로 저장
with open(output_json_path, "w") as f:
    json.dump(coco_data, f, indent=4)

print("finish!")







finish!
